# Implementing a Transformer

In this notebook, you will build an Encoder-Decoder Autoregressive Transformer. While the main classes and structures are already provided, you will gradually implement the core functions of the Transformer architecture.

Before diving into the implementation, take a moment to revisit the core components of Transformers. Keep in mind that we will implement an autoregressive encoder-decoder model — try to think this influences its components.

### 0. Warm-up: A brief practice of torch.nn
Our implementation primarily uses `torch.nn` classes, such as `nn.Linear`, `nn.ReLU`, `nn.LayerNorm`, and `nn.Dropout`. These modules offer essential operations such as fully connected layers (`nn.Linear`), activation functions (`nn.ReLU`), and normalization (`nn.LayerNorm`), forming the core building blocks of our model.

Below are some examples. Feel free to play around with the code and make sure that you understand what the different function calls do.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
torch.manual_seed(33)

In [2]:
my_dim = 20
x = torch.randint(-10,10,(my_dim,)) # Any Random Vector or Matrix
x

tensor([ -2,  -7, -10,   0,   8,   7,  -9,   5,   8,  -1,   5,   8, -10,   8,
         -5,  -3,   3,   7,   5,   2])

In [3]:
my_relu = nn.ReLU()
my_relu(x)

tensor([0, 0, 0, 0, 8, 7, 0, 5, 8, 0, 5, 8, 0, 8, 0, 0, 3, 7, 5, 2])

In [4]:
my_linear = nn.Linear(my_dim, my_dim*3) # Linear Model
# Note: It creates a linear layer that takes as input a vector of size my_dim,
# and outputs a vector of size my_dim*3.
# The learned weight matrix has shape (my_dim*3,my_dim), and the bias is of shape my_dim*3.

y = torch.randn(my_dim, my_dim) # Any Random Vector or Matrix
# Note: It creates a random matrix of shape (my_dim,my_dim).
# Each of its rows is treated as an input vector (of size my_dim)

output = my_linear(y)
# Note: Pass the previous matrix y through the linear layer (row-wise).

print(output.size())
# Note: The output has shape of (my_dim,my_dim*3):
# because each of the my_dim rows is mapped to a my_dim * 3 size vector.

print(output)

torch.Size([20, 60])
tensor([[ 4.6560e-01, -6.4532e-01, -1.1981e-01,  ...,  3.1392e-01,
          8.1311e-01,  5.5818e-02],
        [-4.4255e-01, -3.1730e-01, -5.8750e-01,  ..., -6.7625e-01,
          2.2939e-01,  8.9873e-01],
        [-1.1221e+00,  1.3158e+00, -5.2060e-01,  ...,  2.8394e-01,
         -2.0246e-01, -7.2546e-02],
        ...,
        [-5.4904e-01, -6.2734e-02, -4.5432e-01,  ..., -4.4211e-01,
         -1.4595e-01,  1.5034e-03],
        [ 3.4745e-01, -1.5047e+00, -1.2201e+00,  ..., -1.4794e-01,
         -2.2797e-01,  8.0542e-01],
        [-3.6062e-01,  4.2371e-02, -5.4313e-01,  ..., -5.4947e-02,
          3.7430e-01, -4.7087e-01]], grad_fn=<AddmmBackward0>)


You can learn more about using `torch.nn` in the official PyTorch documentation: https://pytorch.org/docs/stable/nn.html

### 1. Multi Head Attention

We will start by implementing the key component of Transformers -- Multi-Head Attention (MHA).

This class provides a basic implementation of MHA. Your task is to implement the function `scaled_dot_product_attention`. This function calculates the attetntion vectors as we learned in class. Before starting, ensure you understand the other functions in this class, especially `forward`.  You do not need to implement any algebraic functions manually, everything you need is available in `math` and `torch`.

MHA feeds the representations from the previous layer into multiple attention heads, each with its own learned parameter matrices for queries (Q), keys (K), and values (V). Each head independently computes __scaled dot-product__ attention, as outlined below:

**(1)** The initial attention scores are computed as follows:

$attn\_scores = \frac{QK^T}{\sqrt{d_k}}$


Since Q and K are structured with the first dimension as the batch size and the second as the number of heads, ensure you transpose K correctly across the last two dimensions using:

    K.transpose(-2, -1)

**(2)** Next, the mask is applied, and attention probabilities are computed using the softmax function (use `torch.softmax`). Recall that masking is done with 'minus infinity' so that the resulting probability after softmax is effectively 0.

**(3)** Finally, the output is projected onto the V matrix:

$outout = attn\_probes \cdot V$

Ensure the function supports both cases — with and without a mask. If a mask is provided, apply it appropriately. Use `-1e9` as "minus infinity" to exclude specific values. Assume that mask has the correct dimensions, where 0 marks values to be ignored. You'll implement this later.

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "error"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Parameter(torch.Tensor(num_heads, d_model, self.d_k))
        self.W_k = nn.Parameter(torch.Tensor(num_heads, d_model, self.d_k))
        self.W_v = nn.Parameter(torch.Tensor(num_heads, d_model, self.d_k))
        self.W_o = nn.Linear(d_model, d_model)

        self.init_weights()

    def init_weights(self):
        """Initialize weight matrices with Xavier initialization."""
        for param in [self.W_q, self.W_k, self.W_v]:
            nn.init.xavier_uniform_(param)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        Q (torch.Tensor): Query tensor of shape (batch_size, num_heads, seq_length_q,  d_k).
        K (torch.Tensor): Key tensor of shape (batch_size, num_heads, seq_length_v, d_k).
        V (torch.Tensor): Value tensor of shape (batch_size, num_heads, seq_length_v, d_k).
        mask (torch.Tensor): Mask tensor of shape (1, seq_length_q, seq_length_v), where 0
                                       indicates masked positions. Default is None.

        Returns:
        torch.Tensor: The output tensor after applying attention, of shape
                      (batch_size, num_heads, seq_length_q, d_k).
        """
        ## Question 1 - implement this function
        # Compute the attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1))
        attn_scores = attn_scores / math.sqrt(self.d_k)

        # Aplly mask (if exists)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Calculate attention probabilities using softmax
        attn_probes = torch.softmax(attn_scores, dim = -1)

        # Get output by projecting result onto V
        output = torch.matmul(attn_probes, V)
        return output

        ### End of your code

    def forward(self, Q, K, V, mask=None):
        batch_size, seq_len, _ = Q.shape

        # Project each head separately using Einstein summation
        # Q_heads, K_heads, and V_heads each have separate projections per head.
        Q_heads = torch.einsum("bnd,hde->bhne", Q, self.W_q)
        K_heads = torch.einsum("bnd,hde->bhne", K, self.W_k)
        V_heads = torch.einsum("bnd,hde->bhne", V, self.W_v)


        attn_output = self.scaled_dot_product_attention(Q_heads, K_heads, V_heads, mask)
        attn_output = attn_output.transpose(1, 2).reshape(batch_size, seq_len, self.d_model)
        output = self.W_o(attn_output)
        return output


### 2. Feed Forward (MLP) Layer

Now we will be moving to the second building block of Transformers -- the Feed-Forward layers, also known as MLP layers. These layers project each of the input representations into a higher-dimensional space (in our case, we assume `d_ff` is significantly larger than `d_model`). The output is then passed through a non-linear activation function before being projected back to the original model dimension.

**a.** Fill in the dimensions of the weight matrices by replacing the `PLACEHOLDERS` with appropriate class parameters and attributes. Use only predefined attributes, avoiding constant numbers.

**b.** Implement the `forward` function, which processes an input tensor`x` using the Feed-Forward mechanism as described. Utilize the class attributes defined in the `__init__` method.

Recall that the FFN layer operates as follows:

$W_2 \cdot \sigma (W_1 \cdot x)$

σ represents a non-linear function, while W₁ and W₂ are two linear projections.

In [6]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.relu = nn.ReLU()
        ## Question 2a - fill in the dimensions
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        """
        x (torch.Tensor): input tensor of shape (batch_size, seq_length, d_model).
        """
        ## Question 2b - your code here
        # Project input representations into a higher-dimensional space
        h = self.w1(x)

        # Pass through a non-linear activation function
        h = self.relu(h)

        # Project back to the original model dimension
        out = self.w2(h)
        return out

        ## End of your code

### 3. Positional Embeddings

Positional embeddings encode the order of tokens in a sequence, which is essential for models like Transformers that process tokens independently. These embeddings are added to token embeddings and can be either sinusoidal (using sine and cosine functions) or learned as trainable parameters to capture word order.

In this section, __you don't need to make any changes__. Simply run the cell and ensure you understand the code.

In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        # Create a (max_seq_length × d_model) tensor for positional embeddings
        pe = torch.zeros(max_seq_length, d_model)

        # Generate a column vector [0, 1, 2, …, max_seq_length-1]
        # Each row represents the position of a token in the sequence.
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # Compute the scaling factor for each dimension in the embedding.
        # Only every even index (0, 2, 4, …) will be used for sine, the odd ones for cosine.
        # This ensures different frequencies for different dimensions.
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        # Compute sinusoidal embeddings
        pe[:, 0::2] = torch.sin(position * div_term) # even columns
        pe[:, 1::2] = torch.cos(position * div_term) # odd columns

        # Add pe as a non-trainable tensor to the module.
        # (from unsqueeze(0) the shape becomes (1, max_seq_length, d_model) to match batch size.)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # Adds positional embeddings to token embeddings element-wise.
        return x + self.pe[:, :x.size(1)] # (Slices the positional embedding to match the actual sequence length)

### 4. Encoder Layer

In this part, we will use our implementations of MHA and MLP layers to create the Transformer encoder layer. Implement the encoder layer mechanism in the `forward` function. The function should return the tensor after applying all necessary transformations. The argument `x` is the input tensor. We assume that all input sequences have the same length for simplicity, so there is no need to mask out padding tokens.

Remember the residual stream: instead of returning only the updates to the latent tensor, the encoder layer adds these updates and returns an updated version of the input tensor.

Before implementing this section, make sure you understand the usage in `nn.LayerNorm` and `nn.Dropout` class.

Dropout helps preventing overfitting, enhances generalization, and promotes robust feature learning by randomly deactivating neurons during training. This prevents the model from relying too heavily on specific connections, making it more adaptable to new data.

In your code, instead of directly adding the attention output to the residual stream:

    x = x + attn_output

You'll need to apply dropout first:

    x = x + Dropout(attn_output)

Dropout is applied to both the attention output and the FFN output. After the residual connection, you then apply LayerNorm. This follows the Post-Norm approach, where normalization is performed after each sub-layer. Note that other variants exist, such as Pre-Norm, where normalization is applied before the dropout.

In [8]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        """
        x (torch.Tensor): input tensor of shape (batch_size, seq_length, d_model).
        """
        ## Question 4 - your code here
        attn_output = self.self_attn(Q=x,K=x,V=x)

        # Apply Dropout => residual connection => LayerNorm 1
        x = self.norm1(x + self.dropout(attn_output))

        # Feed forward
        ffn_output = self.feed_forward(x)

        # Apply Dropout => residual connection => LayerNorm 2
        x = self.norm2(x + self.dropout(ffn_output))

        return x
        ## End of your code

### 5. Decoder Layer

Implement the decoder layer mechanism in the `forward` function. The function should return the tensor after all necessary transformations. The argument `x` is the decoder input tensor, `enc_output` is the output from the encoder, `no_peek_mask`  ensures that the decoder does not attend to future tokens, preserving the autoregressive nature of the model. You'll implement this mask in the next section, but for now, assume it works as expected.

Be sure to revisit the attention mechanism - make sure you understand the difference between the self-attention and the cross-attention and using them correctly.

In [9]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, enc_output, no_peek_mask):
        """
        x (torch.Tensor): input tensor of shape (batch_size, target_seq_length, d_model).
        enc_output (torch.Tensor): the encoder output, tensor of shape (batch_size, source_seq_length, d_model).
        no_peek_mask (torch.Tensor): the mask for the decoder, tensor of shape (1, target_seq_length, target_seq_length).

        """
        ## Question 5 - your code here
        # Masked self-attention
        attn_output = self.self_attn(Q=x,K=x,V=x,mask=no_peek_mask)

        # Apply Dropout => residual connection => LayerNorm 1
        x = self.norm1(x + self.dropout(attn_output))

        # Cross-attention
        cross_attn_output = self.cross_attn(Q=x,K=enc_output,V=enc_output)

        # Apply Dropout => residual connection => LayerNorm 2
        x = self.norm2(x + self.dropout(cross_attn_output))

        # Feed forward
        ffn_output = self.feed_forward(x)

        # Apply Dropout => residual connection => LayerNorm 3
        x = self.norm3(x + self.dropout(ffn_output))

        return x
        ## End of your code

### 6. Building the Transformer itself!

Now that we have all the building blocks, we can assemble the full Transformer! Review the `__init__` function to see the components we’ve implemented. The forward function is already provided, showing the full pass through the model.

Implement the `generate_mask` function to create the no-peek mask for the decoder, that allows each token to attend only to itself and earlier tokens, blocking future positions. The mask should have a shape of `(1, target_sequence_len, target_sequence_len)`. The value at position (0, i, j) in this tensor should be 1 if token i is allowed to attend to token j, and 0 if it should be masked out.

You might find `torch.ones` and `torch.triu`/`torch.tril` useful for this task.

In [10]:
class CausalTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(CausalTransformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, tgt):
        """
        tgt (torch.Tensor): the target sequence, tensor of shape (batch_size, target_seq_length)

        Returns:
        torch.Tensor: a no peek map for the target_data, of shape
                      (1, seq_length, seq_length).
        """
        ## Question 6 - your code here
        seq_length = tgt.size(1)

        # Create a square matrix of ones
        all_ones_mask = torch.ones((seq_length,seq_length), device=tgt.device)

        # Keep only the lower triangle: diagonal and below it (turn other cells to zeros)
        mask = torch.tril(all_ones_mask)

        # Add a dimention to get the shape: (1, seq_length, seq_length)
        mask = mask.unsqueeze(0)

        return mask
        ## End of your code

    def forward(self, src, tgt):
        np_mask = self.generate_mask(tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, np_mask)

        output = self.fc(dec_output)
        return output

### 7. Testing your Transformer

Run the following cells to test your transformer! This code generates random data and trains the model. Make sure you understand how it works.

__Let it run for 10 training epochs to confirm that you’ve implemented a functioning transformer.__


In [11]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = CausalTransformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [12]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(10):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.682982444763184
Epoch: 2, Loss: 8.552159309387207
Epoch: 3, Loss: 8.46217155456543
Epoch: 4, Loss: 8.386506080627441
Epoch: 5, Loss: 8.325273513793945
Epoch: 6, Loss: 8.2562837600708
Epoch: 7, Loss: 8.182228088378906
Epoch: 8, Loss: 8.088150024414062
Epoch: 9, Loss: 8.009903907775879
Epoch: 10, Loss: 7.925907135009766


Good job!